In [2]:
from RE_model_base import RelationExtractionModel
from RE_model_base import consts
from transformers import AutoTokenizer
import torch
import json

labels_path = "CERED2_LABELS"
id2label = {}
label2id = {}
LABELS = []
with open(labels_path, "r") as f:
    LABELS = json.load(f)
    LABELS = [x[0] + ' ' + x[1] for x in LABELS]
    for i, label in enumerate(LABELS):
        id2label[i] = label
        label2id[label] = i

tokenizer = AutoTokenizer.from_pretrained("stulcrad/XLM-RoBERTa-2", add_prefix_space=True)
model = RelationExtractionModel.from_pretrained("stulcrad/XLM-RoBERTa-2", tokenizer=tokenizer,
                                                id2label=id2label, label2id=label2id,
                                                num_labels=len(LABELS))

sentence1 = "[E1] Jan [/E1] se narodil v [E2] Praze [/E2] ."
sentence2 = "Vydáno bylo 17 . února roku 2017 společností [E2] Domino Records [/E2] , téměř přesně rok po vydání desky [E1] Painting With [/E1] . "
sentence_example = "[E1] Jan Máchal [/E1] ( 9. února 1864, Třebíč [UNK] 15. října 1924, Třebíč ) byl český pedagog, činovník [E2] Sokola [/E2] . "
special_token_dict = {
    '[E1]':consts.E1_START_TOKEN, '[/E1]':consts.E1_END_TOKEN,
    '[E2]':consts.E2_START_TOKEN, '[/E2]':consts.E2_END_TOKEN
}

marked = [token if token not in special_token_dict else special_token_dict[token] for token in sentence1.split(' ')]
marked2 = [token if token not in special_token_dict else special_token_dict[token] for token in sentence2.split(' ')]
marked_example = [token if token not in special_token_dict else special_token_dict[token] for token in sentence_example.split(' ')]
# print(marked)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

marked_sentence = ""
marked_sentence_example = ""
marked_sentence2 = ""
first = True
for token in marked:
    if first:
        first = False
        marked_sentence += token
    else:
        marked_sentence += " " + token
first = True
for token in marked_example:
    if first:
        first = False
        marked_sentence_example += token
    else:
        marked_sentence_example += " " + token
first = True
for token in marked2:
    if first:
        first = False
        marked_sentence2 += token
    else:
        marked_sentence2 += " " + token


inputs = tokenizer(marked_sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
inputs_example = tokenizer(marked_sentence_example, return_tensors="pt", padding=True, truncation=True, max_length=128)
inputs2 = tokenizer(marked_sentence2, return_tensors="pt", padding=True, truncation=True, max_length=128)
inputs.to(device)
inputs_example.to(device)
inputs2.to(device)

with torch.no_grad():
    outputs = model(**inputs)
    outputs_example = model(**inputs_example)
    outputs2 = model(**inputs2)
    logits = outputs["logits"]
    logits_example = outputs_example["logits"]
    logits2 = outputs2["logits"]

pred_labels = logits.argmax().item()
pred_labels_example = logits_example.argmax().item()
pred_labels2 = logits2.argmax().item()
print(id2label[pred_labels])
print(id2label[pred_labels_example])
print(id2label[pred_labels2])
    

P19 místo narození
P463 člen (čeho)
P264 hudební vydavatelství
